In [ ]:
import multiprocessing
print(multiprocessing.cpu_count())

In [ ]:
import os

print(os.cpu_count())

In [1]:
"""
Run DEM tool using multiprocessing
"""

ref   = r'C:\gwork\fireloc\datasets\ref\pt_ref_grid.shp'
dtfld = r'C:\gwork\fireloc\datasets\mdt_m888'
elv_fld = 'ELEVATION'

import os; import datetime as dt
import numpy as np
import multiprocessing as mp
from gasp3.fm          import tbl_to_obj
from gasp3.g.gop.prox  import df_buffer_extent
from gasp3.gt.prop.prj import get_epsg_shp
from gasp3.gt.to.shp import coords_to_boundary
from gasp3.pyt.oss import create_folder
from gasp3.gt.gop.ovlay import clip
from gasp3.pyt.oss import fld_exists
from gasp3.pyt.oss import cpu_cores, lst_ff
from gasp3.df.split import df_split
from gasp3.gd.terrain import make_dem

n_cpu   = cpu_cores()
grid_df = tbl_to_obj(ref)
dfs     = df_split(grid_df, n_cpu)

def dem_thread(df, fld):
    for idx, row in df.iterrows():
        workspace = os.path.join(fld, 'data_' + str(row[cell_id]))
        
        ext_lmt = os.path.join(workspace, 'extra_lmt_{}.shp'.format(str(row[cell_id])))
        alti = os.path.join(workspace,  'alti_{}.shp'.format(str(row[cell_id])))
        
        # Check if dem is already there
        dem_f = 'dem_{}.tif'.format(str(row[cell_id]))
        ff = lst_ff(workspace, file_format='tif', rfilename=True)
        
        if dem_f in ff:
            continue
        
        # Produce dem
        outdem = os.path.join(workspace, dem_f)
        make_dem(workspace, alti, elv_fld, outdem, ext_lmt, 
                 method="CONTOUR", cell_size=10)

thrds = [mp.Process(
    name='th_' + str(i), target=dem_thread, args(dfs[i], dtfld)
) for i in range(len(dfs))]

for t in thrds:
    t.start()

for t in thrds:
    t.join()